In [ ]:
!pip install langchain langchain-community langchain langchain-openai langchain-anthropic chromadb gitpython

# [4주차] 심화과제: 코드 리뷰 LLM 서비스
---

# [MY CODE] 1. 환경 변수 로드

In [453]:
from dotenv import load_dotenv
import os

load_dotenv()

class Config:
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    ANTHROPIC_API_KEY = os.environ['ANTHROPIC_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    CHROMA_DB_PATH = os.environ['CHROMA_DB_PATH']

---
# [MY CODE] 2. LLM(OpenAI, Anthropic)

In [454]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=Config.OPENAI_API_KEY, max_tokens=7000)
anthropic_llm = ChatAnthropic(model_name="claude-3-5-sonnet-20241022", api_key=Config.ANTHROPIC_API_KEY, max_tokens=4000)

---
# [MY CODE] 3. Embedding(OpenAI)

In [455]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=Config.OPENAI_API_KEY)

---
# [MY CODE] 4. VectorStore(ChromaDB)

In [456]:
from langchain.vectorstores import Chroma

vector_store = Chroma(persist_directory=Config.CHROMA_DB_PATH, embedding_function=embeddings)

---
# [MY CODE] 5. Github repository -> Store

리뷰할 깃 레포: https://github.com/adunStudio/codereviewexample

In [457]:
from git import Repo
import hashlib

# GitHub 레포지토리를 로컬로 클론
def clone_github_repo(repo_url, local_path):
    if os.path.exists(local_path):
        print("로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.")
        repo = Repo(local_path)
        repo.remotes.origin.pull()
    else:
        print(f"Cloning {repo_url} into {local_path}...")
        Repo.clone_from(repo_url, local_path)
    print("레포지토리 클론 또는 업데이트 완료!")

# 레포지토리 내 Python 파일 경로를 찾기
def find_python_files(repo_path):
    python_files = []
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files

# 파일의 해시값(SHA256)을 계산
def calculate_file_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

# 파일이 업데이트되었는지 확인
def is_file_updated(file_path, vector_store):
    file_hash = calculate_file_hash(file_path)
    # Chroma에서 해당 파일의 메타데이터 검색
    results = vector_store.similarity_search(file_path, k=1, filter={"file_path": file_path})
    if results:
        # 저장된 해시값과 비교
        return results[0].metadata.get("file_hash") != file_hash
    return True

# Python 파일을 읽어 벡터화하고 Chroma DB에 저장
def store_python_files_in_chroma(file_paths, vector_store):
    for file_path in file_paths:
        try:
            if not is_file_updated(file_path, vector_store):
                print(f"파일이 변경되지 않았습니다. 건너뜁니다: {file_path}")
                continue

            with open(file_path, "r", encoding="utf-8") as file:
                code_content = file.read()

            # 파일 메타데이터 생성
            metadata = {
                "file_path": file_path,
                "file_hash": calculate_file_hash(file_path)
            }

            # 벡터 스토어에 추가
            vector_store.add_texts([code_content], metadatas=[metadata])
            print(f"{file_path} 저장 완료!")
        except Exception as e:
            print(f"파일 처리 중 오류 발생: {file_path}, 오류: {e}")
    vector_store.persist()
    print("Chroma DB에 모든 파일 저장 완료!")

# Chroma DB에 저장된 모든 코드 파일의 메타데이터와 내용을 출력합니다.
def list_all_files_with_content(vector_store):
    try:
        results = vector_store.get(include=["metadatas", "documents"])

        print("저장된 코드 파일과 내용:")
        for i, (metadata, document) in enumerate(zip(results["metadatas"], results["documents"]), 1):
            file_path = metadata.get("file_path", "Unknown")
            print(f"{i}. 파일 경로: {file_path}")
            print("내용:")
            print(document[:300])  # 첫 300자만 출력
            print("-" * 50)
    except Exception as e:
        print(f"전체 파일 목록 및 내용 출력 중 오류 발생: {e}")

# 파일명을 기준으로 Chroma DB에서 내용을 검색하고 반환.
def search_by_filename(file_name, vector_store):
    try:
        # Chroma DB의 모든 데이터를 가져옵니다.
        results = vector_store.get(include=["metadatas", "documents"])

        matching_files = []
        for metadata, document in zip(results["metadatas"], results["documents"]):
            if file_name in metadata.get("file_path", ""):
                matching_files.append((metadata, document))

        if matching_files:
            return matching_files[0][1]
        else:
            print(f"파일명 '{file_name}'이(가) DB에 존재하지 않습니다.")
            return None
    except Exception as e:
        print(f"파일명으로 검색 중 오류 발생: {e}")

In [458]:
github_repo_url = 'https://github.com/adunStudio/codereviewexample'
local_repo_path = "./temp_repo"

# 1. GitHub 레포지토리 클론 또는 업데이트
clone_github_repo(github_repo_url, local_repo_path)

# 2. Python 파일 찾기
python_files = find_python_files(local_repo_path)
print(f"찾은 Python 파일 수: {len(python_files)}")

# 3. Python 파일을 Chroma DB에 저장
store_python_files_in_chroma(python_files, vector_store)

#list_all_files_with_content(vector_store)

로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.
레포지토리 클론 또는 업데이트 완료!
찾은 Python 파일 수: 8
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/world.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/cat.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/text_processor.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/bird.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/animal.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/calculator.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/dog.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/main.py
Chroma DB에 모든 파일 저장 완료!


---
# [MY CODE] 6. Retriever

In [459]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [460]:
# cat.py 소스코드
input_code = search_by_filename('cat.py', vector_store)

# cat.py 소스코드와 연관된 코드 검색
retrievered_codes = retriever.invoke(input_code)

print("\n### 출처 코드 ###")
print(f'문서 개수: {len(retrievered_codes)}')
for code in retrievered_codes:
    print(f"파일: {code.metadata.get('file_path', 'N/A')}")
    print(f"내용: {code.page_content[:200]}")  # 문서 내용 일부 출력
    print('-------' * 5)


### 출처 코드 ###
문서 개수: 5
파일: ./temp_repo/cat.py
내용: from animal import Animal

class Cat(Animal):
    def __init__(self, name, age, color):
        super().__init__(name, age)
        self.color = color
        self.sound = "야옹"

    def scratch(self, 
-----------------------------------
파일: ./temp_repo/dog.py
내용: from animal import Animal

class Dog(Animal):
    def __init__(self, name, age, breed):
        super().__init__(name, age)
        self.breed = breed
        self.sound = "멍멍"

    def fetch(self, it
-----------------------------------
파일: ./temp_repo/animal.py
내용: class Animal:
    def __init__(self, name, age):
        self.name = name
        self.age = age  
        self.sound = None

    def make_sound(self):
        print(f"{self.name}이(가) 소리를 냅니다: {self.s
-----------------------------------
파일: ./temp_repo/bird.py
내용: from animal import Animal

class Bird(Animal):
    def __init__(self, name, age, wing_span):
        super().__init__(name, age)
        self.wing_span =

---
# [MY CODE] 7. PromptTemplates

In [461]:
from langchain.prompts import PromptTemplate
from langchain.prompts import load_prompt

simple_review_prompt = load_prompt('./prompts/simple_review_prompt.json')

input_message = ''

for code in retrievered_codes:
    input_message += f"\n\n[파일명: {code.metadata.get('file_path', 'N/A')}]\n\n"
    input_message += code.page_content

#review_prompt = review_template.format(context=input_message)
#print(review_prompt)

---
# [MY CODE] 8. TransformChain(Retriever -> PromptTemplate)

In [462]:
from langchain.chains import TransformChain

def create_input_code(inputs):
    documents = inputs["documents"]
    input_message = ""
    for doc in documents:
        input_message += f"\n\n[파일명: {doc.metadata.get('file_path', 'N/A')}]\n\n"
        input_message += doc.page_content
    return {"context": input_message}

code_mapping_chain = TransformChain(
    input_variables=["documents"],
    output_variables=["context"],
    transform=create_input_code
)

# [MY CODE] 9. 출력 파서(PydanticOutputParser)

In [463]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class Issue(BaseModel):
    issue: str = Field(description="문제 설명")
    suggestion: str = Field(description="원본 코드 첨부 및 해결 방안")

class FileReview(BaseModel):
    file_name: str = Field(description="파일명")
    issues: List[Issue] = Field(description="발견된 문제 목록")
    summary: str = Field(description="10문장 이내로 요약한 파일 리뷰")

class MultiFileReview(BaseModel):
    reviews: List[FileReview] = Field(description="여러 파일의 리뷰 결과")

pydantic_review_output_parser = PydanticOutputParser(pydantic_object=MultiFileReview)

simple_review_prompt = simple_review_prompt.partial(output_format=pydantic_review_output_parser.get_format_instructions())

---
# [MY CODE] 10. Chain

In [464]:
openai_chain    = retriever | code_mapping_chain | simple_review_prompt | openai_llm    | pydantic_review_output_parser
anthropic_chain = retriever | code_mapping_chain | simple_review_prompt | anthropic_llm | pydantic_review_output_parser

# 입력: cat.py 소스코드
input_code = search_by_filename('cat.py', vector_store)

---
# [MY CODE] 간단 리뷰 서비스
### OpenAI 응답 결과

In [466]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as callback:
    simple_openai_response = openai_chain.invoke(input_code)

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 1228
응답 토큰 수: 641
총 토큰 수: 1869
총 비용: $0.0004919999999999999


In [467]:
import json

# print(simple_openai_response)
# print(json.dumps(simple_openai_response, indent=2, ensure_ascii=False))
review_result = simple_openai_response
# 결과 출력
print("리뷰 결과:")
for review in review_result.reviews:
    print(f"파일명: {review.file_name}")
    for issue in review.issues:
        print(f"- 문제: {issue.issue}")
        print(f"  해결 방안: {issue.suggestion}")
    print(f"요약: {review.summary}")
    print("-" * 50)

리뷰 결과:
파일명: ./temp_repo/cat.py
- 문제: 문자열 포매팅 방식이 일관되지 않음.
  해결 방안: f-string을 사용하여 코드를 일관되게 만듭니다: print(f'{self.name}이(가) {item}을(를) 긁습니다.')
- 문제: item 변수의 접근 방식이 잘못됨.
  해결 방안: self.item 대신 item을 사용하여 올바른 변수를 참조합니다.
요약: Cat 클래스는 문자열 포매팅의 비일관성과 잘못된 변수 참조 문제가 있습니다.
--------------------------------------------------
파일명: ./temp_repo/dog.py
- 문제: fetch 메소드에 불필요한 루프가 존재.
  해결 방안: 루프를 제거하고 바로 print 문으로 대체하면 됩니다.
요약: Dog 클래스의 fetch 메소드는 비효율적인 루프를 가지고 있어 성능 저하를 야기합니다.
--------------------------------------------------
파일명: ./temp_repo/animal.py
- 문제: make_sound 메소드가 단순히 소리만 출력.
  해결 방안: 소리를 보다 의미 있게 사용할 수 있도록 시스템 내 다른 메소드에서 활용하세요.
요약: Animal 클래스의 make_sound 메소드는 개선의 여지가 있으며, 더 나은 활용 방안을 모색해야 합니다.
--------------------------------------------------
파일명: ./temp_repo/bird.py
- 문제: 날아오르는 메시지에서 '!'의 중복 사용.
  해결 방안: 중복된 느낌을 주지 않도록 '!'를 한 번만 사용합니다: print(f'{self.name}이(가) 날개 길이 {self.wing_span}cm로 날아오릅니다!')
요약: Bird 클래스의 fly 메소드는 출력에서 느낌표의 중복 사용을 피해야 합니다.
-----------------------------------------------

### Anthropic 응답 결과

In [468]:
simple_anthropic_response = anthropic_chain.invoke(input_code)
#print(simple_anthropic_response.content)
#print(json.dumps(simple_anthropic_response, indent=2, ensure_ascii=False))
review_result = simple_anthropic_response
# 결과 출력
print("리뷰 결과:")
for review in review_result.reviews:
    print(f"파일명: {review.file_name}")
    for issue in review.issues:
        print(f"- 문제: {issue.issue}")
        print(f"  해결 방안: {issue.suggestion}")
    print(f"요약: {review.summary}")
    print("-" * 50)

리뷰 결과:
파일명: ./temp_repo/animal.py
- 문제: sound 속성이 None으로 초기화되어 make_sound() 메서드에서 NoneType 에러가 발생할 수 있습니다.
  해결 방안: sound 속성에 기본값을 문자열로 설정하거나, make_sound() 메서드에서 None 체크를 추가하세요:
```python
def make_sound(self):
    if self.sound is None:
        print(f"{self.name}이(가) 소리를 내지 않습니다.")
    else:
        print(f"{self.name}이(가) 소리를 냅니다: {self.sound}")
```
- 문제: age 파라미터의 유효성 검사가 없습니다.
  해결 방안: 나이에 대한 유효성 검사를 추가하세요:
```python
def __init__(self, name, age):
    if not isinstance(age, (int, float)) or age < 0:
        raise ValueError("나이는 0 이상의 숫자여야 합니다.")
    self.name = name
    self.age = age
```
요약: Animal 클래스는 기본적인 동물의 특성을 정의한 기초 클래스입니다. 전반적으로 단순하고 명확하나, 일부 개선이 필요한 부분이 있습니다.
--------------------------------------------------
파일명: ./temp_repo/cat.py
- 문제: scratch 메서드에서 self.item을 참조하나, 해당 속성이 정의되어 있지 않습니다.
  해결 방안: 올바른 변수를 사용하도록 수정하세요:
```python
def scratch(self, item):
    if not item:
        print(f"{self.name}이(가) 긁을 것이 없습니다!")
    else:
        print(f"{self.name}이(가) {item}을(를) 긁습니

---
# [MY CODE] 프리미엄 코딩 리뷰 서비스
### 5개 항목 심화 코딩 리뷰 (비동기 방식 사용)
- 코딩 컨벤션
- 설계(유지보수, SOLID)
- 퍼포먼스
- 바퀴재발명
- ddd/tdd 방법론

프롬프트 위치: https://github.com/adunStudio/hanghae99_plusAI_2/blob/main/5%EC%A3%BC%EC%B0%A8/prompt_maker.ipynb

5개 항목별 사용할 프롬프트와 LLM Chain 셋팅

In [498]:
premium_service = ['coding_convention', 'ddd_tdd', 'maintainability', 'performance', 'reinvention']
premium_llms    = [openai_llm, openai_llm, anthropic_llm, openai_llm, anthropic_llm]

output_format = pydantic_review_output_parser.get_format_instructions()

premium_prompts = [
    load_prompt(f'./prompts/{service}_prompt.json')
    .partial(output_format=output_format)
    for service in premium_service
]

premium_chains = [
    retriever | code_mapping_chain | prompt | llm | pydantic_review_output_parser
    for llm, prompt in zip(premium_llms, premium_prompts)
]

프롬프트별 독립적이므로 시퀀스는 X, 배치 방식은 모두 완료 후 보여줘 답답~ 하므로 비동기 방식 사용

In [499]:
import asyncio

async def invoke_async(service_name, chain, input_data):
    review_result = await chain.ainvoke(input_data)
    print(f"{service_name} 리뷰 결과:")
    for review in review_result.reviews:
        print(f"파일명: {review.file_name}")
        for issue in review.issues:
            print(f"- 문제: {issue.issue}")
            print(f"  해결 방안: {issue.suggestion}")
        print(f"요약: {review.summary}")
        print("-" * 50)

async def invoke_parallel(chains, input_data):
    tasks = [invoke_async(premium_service[idx], chain, input_data) for idx, chain in enumerate(chains)]
    await asyncio.gather(*tasks)

input_code = search_by_filename('cat.py', vector_store)

print("프리미엄 코드 리뷰 시작")
await invoke_parallel(premium_chains, input_code)
print("프리미엄 코드 리뷰 완료")

프리미엄 코드 리뷰 시작
reinvention 리뷰 결과:
파일명: ./temp_repo/cat.py
- 문제: 문자열 포매팅 방식이 섞여 있음 (format vs f-string)
  해결 방안: 모든 문자열 포매팅을 f-string으로 통일:
```python
print(f"{self.name}이(가) {item}을(를) 긁습니다.")
```
- 문제: scratch 메서드에서 item 변수명 오류
  해결 방안: self.item을 item으로 수정:
```python
print(f"{self.name}이(가) {item}을(를) 긁습니다.")
```
요약: Cat 클래스는 문자열 포매팅 방식이 일관되지 않고, scratch 메서드에서 버그가 있습니다.
--------------------------------------------------
파일명: ./temp_repo/dog.py
- 문제: fetch 메서드의 불필요한 지연 루프
  해결 방안: 불필요한 for 루프 제거:
```python
def fetch(self, item):
    print(f"{self.name}이(가) {item}을(를) 가져옵니다!")
```
요약: Dog 클래스에서 불필요한 지연 로직이 있습니다.
--------------------------------------------------
파일명: ./temp_repo/bird.py
- 문제: fly 메서드에서 불필요한 문자열 연산 사용
  해결 방안: 단순 f-string으로 수정:
```python
print(f"{self.name}이(가) 날개 길이 {self.wing_span}cm로 날아오릅니다!")
```
요약: Bird 클래스에서 문자열 연산이 불필요하게 사용되었습니다.
--------------------------------------------------
파일명: ./temp_repo/world.py
- 문제: isinstance 체크 대신 더 파이썬스러운 방식 사용 가능
  해결 방안: EAFP(Easi